In [3]:
import numpy as np
import pandas as pd

import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress 
from scipy import stats
import scipy.stats as sts
# import hvplot.pandas

import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
# Import data
combined_news_df = pd.read_csv('../Data/Alicia/Combined-NewsData-CSV.csv')
combined_news_df.head()


,Title,Article,Headline,Title_URL,Image,Facebook_likes,Facebook_comments,Facebook_shares,Comment_replies
0,CNN,Po Murray writes that the two mass killings in...,Opinion: What has to happen after the Colorado...,https://www.facebook.com/cnn/?__cft__[0]=AZVAV...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,102,98 Comments,6 Shares,1 Reply
1,CNN,Mississippi is one of three states that have e...,More states aim to offer Covid-19 vaccines to ...,https://www.facebook.com/cnn/?__cft__[0]=AZU2o...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,574,98 Comments,31 Shares,1 Reply
2,CNN,"In 2018, the city of Boulder, Colorado, passed...",Boulder banned assault weapons in 2018. A judg...,https://www.facebook.com/cnn/?__cft__[0]=AZVNa...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,1.1K,1.2K Comments,119 Shares,112 Replies
3,CNN,"""The life of a single parent is challenging to...",Opinion: My Covid life as a single mom: Like j...,https://www.facebook.com/cnn/?__cft__[0]=AZWhO...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,554,202 Comments,37 Shares,9 Replies
4,CNN,British people attempting to go on vacation wh...,"New UK law means $7,000 fines for vacations ab...",https://www.facebook.com/cnn/?__cft__[0]=AZXIu...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,912,390 Comments,83 Shares,View more comments


In [3]:
# Split the values to show only numeric without the words like Shares, Comments, Replies
combined_news_df["Facebook_likes"] = combined_news_df["Facebook_likes"].str.split(" ", n = 1, expand = True)
combined_news_df["Facebook_comments"] = combined_news_df["Facebook_comments"].str.split(" ", n = 1, expand = True)
combined_news_df["Facebook_shares"] = combined_news_df["Facebook_shares"].str.split(" ", n = 1, expand = True)
combined_news_df["Comment_replies"] = combined_news_df["Comment_replies"].str.split(" ", n = 1, expand = True)

In [4]:
combined_news_df

,Title,Article,Headline,Title_URL,Image,Facebook_likes,Facebook_comments,Facebook_shares,Comment_replies
0,CNN,Po Murray writes that the two mass killings in...,Opinion: What has to happen after the Colorado...,https://www.facebook.com/cnn/?__cft__[0]=AZVAV...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,102,98,6,1
1,CNN,Mississippi is one of three states that have e...,More states aim to offer Covid-19 vaccines to ...,https://www.facebook.com/cnn/?__cft__[0]=AZU2o...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,574,98,31,1
2,CNN,"In 2018, the city of Boulder, Colorado, passed...",Boulder banned assault weapons in 2018. A judg...,https://www.facebook.com/cnn/?__cft__[0]=AZVNa...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,1.1K,1.2K,119,112
3,CNN,"""The life of a single parent is challenging to...",Opinion: My Covid life as a single mom: Like j...,https://www.facebook.com/cnn/?__cft__[0]=AZWhO...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,554,202,37,9
4,CNN,British people attempting to go on vacation wh...,"New UK law means $7,000 fines for vacations ab...",https://www.facebook.com/cnn/?__cft__[0]=AZXIu...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,912,390,83,View
...,...,...,...,...,...,...,...,...,...
1040,Fox News,BREAKING: The IRS may have decided to heed the...,IRS planning to delay tax filing deadline unti...,https://www.facebook.com/FoxNews/?__cft__[0]=A...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,1.1K,588,314,View
1041,Fox News was live.,White House press secretary Jen Psaki holds a ...,NaN,https://www.facebook.com/FoxNews/?__cft__[0]=A...,NaN,5.8K,12K,461,View
1042,Fox News was live.,Federal Reserve Chairman Jerome Powell holds a...,NaN,https://www.facebook.com/FoxNews/?__cft__[0]=A...,NaN,1.6K,2.9K,275,7
1043,Fox News,NaN,"Fed holds rates near zero, lifts growth outloo...",https://www.facebook.com/FoxNews/?__cft__[0]=A...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,890,619,91,View


In [5]:
# To replace the "view" text from the column "Comment_replies" with zero 
combined_news_df["Comment_replies"] = combined_news_df["Comment_replies"].replace(to_replace ="View", value ="0")

In [6]:
combined_news_df.head()

,Title,Article,Headline,Title_URL,Image,Facebook_likes,Facebook_comments,Facebook_shares,Comment_replies
0,CNN,Po Murray writes that the two mass killings in...,Opinion: What has to happen after the Colorado...,https://www.facebook.com/cnn/?__cft__[0]=AZVAV...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,102,98,6,1
1,CNN,Mississippi is one of three states that have e...,More states aim to offer Covid-19 vaccines to ...,https://www.facebook.com/cnn/?__cft__[0]=AZU2o...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,574,98,31,1
2,CNN,"In 2018, the city of Boulder, Colorado, passed...",Boulder banned assault weapons in 2018. A judg...,https://www.facebook.com/cnn/?__cft__[0]=AZVNa...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,1.1K,1.2K,119,112
3,CNN,"""The life of a single parent is challenging to...",Opinion: My Covid life as a single mom: Like j...,https://www.facebook.com/cnn/?__cft__[0]=AZWhO...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,554,202,37,9
4,CNN,British people attempting to go on vacation wh...,"New UK law means $7,000 fines for vacations ab...",https://www.facebook.com/cnn/?__cft__[0]=AZXIu...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,912,390,83,0


In [7]:
# Convert the "K" to thousands in numerical 
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000
    return x


In [8]:
combined_news_df["Facebook_comments"] = combined_news_df["Facebook_comments"].apply(value_to_float)
combined_news_df["Facebook_likes"] = combined_news_df["Facebook_likes"].apply(value_to_float)
combined_news_df["Facebook_shares"] = combined_news_df["Facebook_shares"].apply(value_to_float)


In [9]:
combined_news_df

,Title,Article,Headline,Title_URL,Image,Facebook_likes,Facebook_comments,Facebook_shares,Comment_replies
0,CNN,Po Murray writes that the two mass killings in...,Opinion: What has to happen after the Colorado...,https://www.facebook.com/cnn/?__cft__[0]=AZVAV...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,102,98,6,1
1,CNN,Mississippi is one of three states that have e...,More states aim to offer Covid-19 vaccines to ...,https://www.facebook.com/cnn/?__cft__[0]=AZU2o...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,574,98,31,1
2,CNN,"In 2018, the city of Boulder, Colorado, passed...",Boulder banned assault weapons in 2018. A judg...,https://www.facebook.com/cnn/?__cft__[0]=AZVNa...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,1100.0,1200.0,119,112
3,CNN,"""The life of a single parent is challenging to...",Opinion: My Covid life as a single mom: Like j...,https://www.facebook.com/cnn/?__cft__[0]=AZWhO...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,554,202,37,9
4,CNN,British people attempting to go on vacation wh...,"New UK law means $7,000 fines for vacations ab...",https://www.facebook.com/cnn/?__cft__[0]=AZXIu...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,912,390,83,0
...,...,...,...,...,...,...,...,...,...
1040,Fox News,BREAKING: The IRS may have decided to heed the...,IRS planning to delay tax filing deadline unti...,https://www.facebook.com/FoxNews/?__cft__[0]=A...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,1100.0,588,314,0
1041,Fox News was live.,White House press secretary Jen Psaki holds a ...,NaN,https://www.facebook.com/FoxNews/?__cft__[0]=A...,NaN,5800.0,12000.0,461,0
1042,Fox News was live.,Federal Reserve Chairman Jerome Powell holds a...,NaN,https://www.facebook.com/FoxNews/?__cft__[0]=A...,NaN,1600.0,2900.0,275,7
1043,Fox News,NaN,"Fed holds rates near zero, lifts growth outloo...",https://www.facebook.com/FoxNews/?__cft__[0]=A...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,890,619,91,0


In [10]:
# Convert to new csv file
cleaned_df = combined_news_df.to_csv("cleaned_data")
cleaned_df


In [6]:
# Import data
fb1_df = pd.read_csv('../Data/Alicia/fb1_test_data.csv')
fb1_df.head()


,Unnamed: 0,Title,Article,Headline,Title_URL,Image,Facebook_likes,Facebook_comments,Facebook_shares,Comment_replies,results
0,0,CNN,Po Murray writes that the two mass killings in...,Opinion: What has to happen after the Colorado...,https://www.facebook.com/cnn/?__cft__[0]=AZVAV...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,102,98 Comments,6 Shares,1 Reply,'REAL'
1,1,CNN,Mississippi is one of three states that have e...,More states aim to offer Covid-19 vaccines to ...,https://www.facebook.com/cnn/?__cft__[0]=AZU2o...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,574,98 Comments,31 Shares,1 Reply,'FAKE'
2,2,CNN,"In 2018, the city of Boulder, Colorado, passed...",Boulder banned assault weapons in 2018. A judg...,https://www.facebook.com/cnn/?__cft__[0]=AZVNa...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,1.1K,1.2K Comments,119 Shares,112 Replies,'FAKE'
3,3,CNN,"""The life of a single parent is challenging to...",Opinion: My Covid life as a single mom: Like j...,https://www.facebook.com/cnn/?__cft__[0]=AZWhO...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,554,202 Comments,37 Shares,9 Replies,'FAKE'
4,4,CNN,British people attempting to go on vacation wh...,"New UK law means $7,000 fines for vacations ab...",https://www.facebook.com/cnn/?__cft__[0]=AZXIu...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,912,390 Comments,83 Shares,View more comments,'FAKE'


In [7]:
# Import data
fb2_df = pd.read_csv('../Data/Alicia/fb2_test_data.csv')
fb2_df.head()


,Unnamed: 0,Title,Article,Headline,Title_URL,Image,Facebook_likes,Facebook_comments,Facebook_shares,Comment_replies,results
0,0,Conservative Post,"This program, and the $500 monthly checks it o...",Struggling White Families Not Allowed To Parte...,https://www.facebook.com/heyconservativepost/?...,https://l.facebook.com/l.php?u=https%3A%2F%2Fc...,180,11 Comments,29 Shares,View 40 more comments,'FAKE'
1,1,Conservative Post,Biden appeared to simply give up on what he wa...,The First TV: Biden Lapses Into Nonsense Durin...,https://www.facebook.com/heyconservativepost/?...,https://l.facebook.com/l.php?u=https%3A%2F%2Fc...,220,7 Comments,391 Shares,View previous comments,'FAKE'
2,2,Conservative Post,Right observation! Biden used notes extensivel...,Fox’s Baier: I Haven’t Seen Presidents Flip Th...,https://www.facebook.com/heyconservativepost/?...,https://l.facebook.com/l.php?u=https%3A%2F%2Fc...,128,43 Comments,2.6K Shares,View previous comments,'FAKE'
3,3,Conservative Post,She has been charged with felony obstruction o...,Georgia Dem Arrested After Defying Police and ...,https://www.facebook.com/heyconservativepost/?...,https://l.facebook.com/l.php?u=https%3A%2F%2Fc...,364,64 Comments,43 Shares,View previous comments,'FAKE'
4,4,Conservative Post,Let's go Georgia!,Georgia Governor Signs Into Law Sweeping Elect...,https://www.facebook.com/heyconservativepost/?...,https://l.facebook.com/l.php?u=https%3A%2F%2Fc...,845,58 Comments,1.1K Shares,View previous comments,'REAL'


In [9]:
all_news_df = pd.concat([fb1_df, fb2_df])
all_news_df

,Unnamed: 0,Title,Article,Headline,Title_URL,Image,Facebook_likes,Facebook_comments,Facebook_shares,Comment_replies,results
0,0,CNN,Po Murray writes that the two mass killings in...,Opinion: What has to happen after the Colorado...,https://www.facebook.com/cnn/?__cft__[0]=AZVAV...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,102,98 Comments,6 Shares,1 Reply,'REAL'
1,1,CNN,Mississippi is one of three states that have e...,More states aim to offer Covid-19 vaccines to ...,https://www.facebook.com/cnn/?__cft__[0]=AZU2o...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,574,98 Comments,31 Shares,1 Reply,'FAKE'
2,2,CNN,"In 2018, the city of Boulder, Colorado, passed...",Boulder banned assault weapons in 2018. A judg...,https://www.facebook.com/cnn/?__cft__[0]=AZVNa...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,1.1K,1.2K Comments,119 Shares,112 Replies,'FAKE'
3,3,CNN,"""The life of a single parent is challenging to...",Opinion: My Covid life as a single mom: Like j...,https://www.facebook.com/cnn/?__cft__[0]=AZWhO...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,554,202 Comments,37 Shares,9 Replies,'FAKE'
4,4,CNN,British people attempting to go on vacation wh...,"New UK law means $7,000 fines for vacations ab...",https://www.facebook.com/cnn/?__cft__[0]=AZXIu...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,912,390 Comments,83 Shares,View more comments,'FAKE'
...,...,...,...,...,...,...,...,...,...,...,...
934,935,BuzzFeed Politics,"Joe Biden's announcement, part of a national a...",White House: All US Adults Will Be Eligible Fo...,https://www.facebook.com/BuzzFeedPol/,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,1.1K,588 Comments,314 Shares,View previous comments,'REAL'
935,936,BuzzFeed Politics,“We will move with a sense of urgency on clima...,Michael Regan Will Be The First Black Man To L...,https://www.facebook.com/BuzzFeedPol/,NaN,5.8K,12K Comments,461 Shares,View 9 more replies,'FAKE'
936,937,BuzzFeed Politics,Garland has said investigating the Capitol ins...,The Senate Confirmed Merrick Garland As The Ne...,https://www.facebook.com/BuzzFeedPol/,NaN,1.6K,2.9K Comments,275 Shares,7 Replies,'REAL'
937,938,BuzzFeed Politics,"Checks of $1,400, unemployment benefits, and h...",Congress Just Passed Another Round Of COVID Ch...,https://www.facebook.com/BuzzFeedPol/,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,890,619 Comments,91 Shares,View previous comments,'REAL'


In [12]:
# Drop first column
all_news_df = all_news_df.drop(all_news_df.columns[0], axis=1)
all_news_df

,Title,Article,Headline,Title_URL,Image,Facebook_likes,Facebook_comments,Facebook_shares,Comment_replies,results
0,CNN,Po Murray writes that the two mass killings in...,Opinion: What has to happen after the Colorado...,https://www.facebook.com/cnn/?__cft__[0]=AZVAV...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,102,98 Comments,6 Shares,1 Reply,'REAL'
1,CNN,Mississippi is one of three states that have e...,More states aim to offer Covid-19 vaccines to ...,https://www.facebook.com/cnn/?__cft__[0]=AZU2o...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,574,98 Comments,31 Shares,1 Reply,'FAKE'
2,CNN,"In 2018, the city of Boulder, Colorado, passed...",Boulder banned assault weapons in 2018. A judg...,https://www.facebook.com/cnn/?__cft__[0]=AZVNa...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,1.1K,1.2K Comments,119 Shares,112 Replies,'FAKE'
3,CNN,"""The life of a single parent is challenging to...",Opinion: My Covid life as a single mom: Like j...,https://www.facebook.com/cnn/?__cft__[0]=AZWhO...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,554,202 Comments,37 Shares,9 Replies,'FAKE'
4,CNN,British people attempting to go on vacation wh...,"New UK law means $7,000 fines for vacations ab...",https://www.facebook.com/cnn/?__cft__[0]=AZXIu...,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,912,390 Comments,83 Shares,View more comments,'FAKE'
...,...,...,...,...,...,...,...,...,...,...
934,BuzzFeed Politics,"Joe Biden's announcement, part of a national a...",White House: All US Adults Will Be Eligible Fo...,https://www.facebook.com/BuzzFeedPol/,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,1.1K,588 Comments,314 Shares,View previous comments,'REAL'
935,BuzzFeed Politics,“We will move with a sense of urgency on clima...,Michael Regan Will Be The First Black Man To L...,https://www.facebook.com/BuzzFeedPol/,NaN,5.8K,12K Comments,461 Shares,View 9 more replies,'FAKE'
936,BuzzFeed Politics,Garland has said investigating the Capitol ins...,The Senate Confirmed Merrick Garland As The Ne...,https://www.facebook.com/BuzzFeedPol/,NaN,1.6K,2.9K Comments,275 Shares,7 Replies,'REAL'
937,BuzzFeed Politics,"Checks of $1,400, unemployment benefits, and h...",Congress Just Passed Another Round Of COVID Ch...,https://www.facebook.com/BuzzFeedPol/,https://external-sjc3-1.xx.fbcdn.net/safe_imag...,890,619 Comments,91 Shares,View previous comments,'REAL'
